<center><img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks<a class="jp-InternalAnchorLink" href="#unSkript-Runbooks" target="_self">&para;</a></h1>
<div class="alert alert-block alert-success">
<h3 id="Objective">Objective<a class="jp-InternalAnchorLink" href="#Objective" target="_self">&para;</a></h3>
<br><strong style="color: #000000;"><em>Find and Delete unused NAT Gateways</em></strong></div>
</center>
<p>&nbsp;</p>
<center>
<h2 id="Delete-Unused-AWS-Log-Streams"><u>Delete Unuse</u><span style="text-decoration: underline;">d <strong style="color: rgb(0, 0, 0); text-decoration: underline;">N</strong><strong style="color: rgb(0, 0, 0); text-decoration: underline;">AT Gateways</strong></span></h2>
</center>
<h1 id="Steps-Overview">Steps Overview</h1>
<p>1)<a href="#1" target="_self" rel="noopener"> Find unused NAT gateways</a><br>2)<a href="#2" target="_self" rel="noopener"> Delete unused NAT gateways</a></p>

In [ ]:
if nat_gateway_ids and not region:
    raise SystemExit("Provide a region for the NAT Gateway ID's!")

<h3 id="List-all-AWS-Regions">List all AWS Regions</h3>
<p>This action fetches all AWS Regions to execute Step 1👇. This action will only execute if no <code>region</code> is provided.</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following ouput: <code>region</code></p>
</blockquote>

In [7]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from typing import Dict, List
import pprint

from beartype import beartype
@beartype
def aws_list_all_regions_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_list_all_regions(handle) -> List:
    """aws_list_all_regions lists all the AWS regions

        :type handle: object
        :param handle: Object returned from Task Validate

        :rtype: Result List of result
    """

    result = handle.aws_cli_command("aws ec2 --region us-west-2 describe-regions --all-regions --query 'Regions[].{Name:RegionName}' --output text")
    if result is None or result.returncode != 0:
        print("Error while executing command : {}".format(result))
        return str()
    result_op = list(result.stdout.split("\n"))
    list_region = [x for x in result_op if x != '']
    return list_region


task = Task(Workflow())
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not region",
    "condition_result": true
    }''')

task.configure(outputName="region")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_all_regions, lego_printer=aws_list_all_regions_printer, hdl=hdl, args=args)

Error in obtaining credential: get_secret() missing 1 required positional argument: 'key'


<h3 id="Filter-unused-log-streams"><a id="1" target="_self" rel="nofollow"></a>Filter unused NAT Gateways</h3>
<p>Using unSkript's Filter AWS Find Unused NAT Gateways action, we will find unused gateways given a threshold number of days from the metric <span style="color: rgb(53, 152, 219);">BytesIn</span>. If the metric gives an empty result, we consider the NAT Gateway to be unused in the last <em>x days.</em></p>
<blockquote>
<p>This action takes the following parameters: <code>region, threhold_days</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>unused_gateways</code></p>
</blockquote>

In [ ]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import Optional, Tuple
from pydantic import BaseModel, Field
from datetime import datetime, timedelta
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
import pprint

from beartype import beartype
@beartype
def aws_filter_unused_nat_gateway_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def is_nat_gateway_used(handle, nat_gateway, start_time, end_time):
    datapoints = []
    if nat_gateway['State'] != 'deleted':
        # Get the metrics data for the specified NAT Gateway over the last 7 days
        metrics_data = handle.get_metric_statistics(
            Namespace='AWS/NATGateway',
            MetricName='BytesIn',
            Dimensions=[
                {
                    'Name': 'NatGatewayId',
                    'Value': nat_gateway['NatGatewayId']
                },
            ],
            StartTime=start_time,
            EndTime=end_time,
            Period=3600,
            Statistics=['Sum']
        )
        datapoints += metrics_data['Datapoints']
    if len(datapoints) == 0 or metrics_data['Datapoints'][0]['Sum']:
        return False
    else:
        return True


@beartype
def aws_filter_unused_nat_gateway(handle, number_of_days: int = 7, region: str = "") -> Tuple:
    """aws_get_natgateway_by_vpc Returns an array of NAT gateways.

        :type region: string
        :param region: Region to filter NAT Gateways.

        :type number_of_days: int
        :param number_of_days: Number of days to check the Datapoints.

        :rtype: Array of NAT gateways.
    """
    result = []
    end_time = datetime.utcnow()
    start_time = end_time - timedelta(days=number_of_days)
    all_regions = [region]
    if not region:
        all_regions = aws_list_all_regions(handle)

    for reg in all_regions:
        try:
            ec2Client = handle.client('ec2', region_name=reg)
            cloudwatch = handle.client('cloudwatch', region_name=reg)
            response = ec2Client.describe_nat_gateways()
            for nat_gateway in response['NatGateways']:
                nat_gateway_info = {}
                if not is_nat_gateway_used(cloudwatch, nat_gateway, start_time, end_time):
                    nat_gateway_info["nat_gateway_id"] = nat_gateway['NatGatewayId']
                    nat_gateway_info["reg"] = reg
                    result.append(nat_gateway_info)
        except Exception as e:
            pass

    if len(result) != 0:
        return (False, result)
    else:
        return (True, None)


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "number_of_days": "int(threshold_days)",
    "region": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "region",
    "iter_parameter": "region"
    }''')

task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not nat_gateway_ids",
    "condition_result": true
    }''')
task.configure(outputName="unused_gateways")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_filter_unused_nat_gateway, lego_printer=aws_filter_unused_nat_gateway_printer, hdl=hdl, args=args)

<h3 id="Create-List-of-Unused-Log-Streams&para;">Create List of Unused NAT Gateways</h3>
<p>This action filters regions that have no unused gateways and creates a list of those that have them.</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>all_unused_gateways</code></p>
</blockquote>

In [ ]:
all_unused_gateways = []
dummy = []
try:
    for reg,res in unused_gateways.items():
        if res[0]==False:
            if len(res[1])!=0:
                dummy = res[1]
                for x in dummy:
                    all_unused_gateways.append(x)
except Exception:
    for nat_id in nat_gateway_ids:
        data_dict = {}
        data_dict["reg"] = region[0]
        data_dict["nat_gateway_id"] = nat_id
        all_unused_gateways.append(data_dict)
print(all_unused_gateways)
task.configure(outputName="all_unused_gateways")

<h3 id="Delete-unused-log-streams"><a id="2" target="_self" rel="nofollow"></a>Delete unused NAT Gateways</h3>
<p>This action deleted unused log streams found in Step 1.&nbsp;</p>
<blockquote>
<p>This action takes the following parameters:&nbsp;<code>region, nat_gateway_id</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Dict
import pprint


from beartype import beartype
@beartype
def aws_delete_nat_gateway_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_delete_nat_gateway(handle, nat_gateway_id: str, region: str) -> Dict:
    """aws_delete_nat_gateway Returns an dict of NAT gateways information.

        :type region: string
        :param region: Region to filter instances.

        :type nat_gateway_id: string
        :param nat_gateway_id: ID of the NAT Gateway.

        :rtype: dict of NAT gateways information.
    """
    try:
        ec2Client = handle.client('ec2', region_name=region)
        response = ec2Client.delete_nat_gateway(NatGatewayId=nat_gateway_id)
        return response
    except Exception as e:
        raise Exception(e)


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "nat_gateway_id": "iter.get(\\"nat_gateway_id\\")",
    "region": "iter.get(\\"reg\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "all_unused_gateways",
    "iter_parameter": ["nat_gateway_id","region"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(all_unused_gateways)!=0",
    "condition_result": true
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_delete_nat_gateway, lego_printer=aws_delete_nat_gateway_printer, hdl=hdl, args=args)

<h3 id="Conclusion">Conclusion<a class="jp-InternalAnchorLink" href="#Conclusion" target="_self">&para;</a></h3>
<p>In this Runbook, we were able to filter unused NAT Gateways given a threshold number of days and delete them. This runbook enables us to saves cost as AWS charges us based on the number of hours the gateway was available and the data (GB) it processes. To view the full platform capabilities of unSkript please visit&nbsp;<a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>